bow balanced LR

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load dataset
import pandas as pd
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [normalize_word(w) for w in text_tokens if w not in stop_words]
    return " ".join(filtered_text)

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)

def stemming(data):
    return " ".join([stemmer.stem(word) for word in data.split()])

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

# Upsample minority class
minority_upsampled = resample(minority_class, replace=True,n_samples=len(majority_class),random_state=35)

text_df = pd.concat([majority_class, minority_upsampled])

#  TF-IDF
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
logreg_model = LogisticRegression(class_weight='balanced', solver='liblinear')  # Set class_weight to balanced
logreg_model.fit(x_train, y_train)
logreg_pred = logreg_model.predict(x_test)
logreg_acc = accuracy_score(y_test, logreg_pred)
f1_macro = f1_score(y_test, logreg_pred, average='macro')

print("Test accuracy: {:.2f}%".format(logreg_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],
}

grid = GridSearchCV(LogisticRegression(class_weight='balanced', solver='liblinear'), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)
print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Logistic Regression - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_logreg_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Logistic Regression - F1 Macro score: {:.2f}".format(best_logreg_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(lambda x: stemming(x))
evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/bow-balance-logistic-regression.csv', index=False)

bow balanced naive bayse

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load dataset
import pandas as pd
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    return " ".join(filtered_text)

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

# Upsample minority class
minority_upsampled = resample(minority_class,replace=True,n_samples=len(majority_class),random_state=35)

text_df = pd.concat([majority_class, minority_upsampled])

# TF-IDF
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)
nb_pred = nb_model.predict(x_test)
nb_acc = accuracy_score(y_test, nb_pred)
f1_macro = f1_score(y_test, nb_pred, average='macro')
print("Test accuracy: {:.2f}%".format(nb_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, nb_pred))
print("\n")
print(classification_report(y_test, nb_pred))

# Hyperparameter tuning
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5]
}

grid = GridSearchCV(MultinomialNB(), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)
print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

nb_acc = accuracy_score(y_test, y_pred)
print("Best Naive Bayes - Test accuracy: {:.2f}%".format(nb_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_nb_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Naive Bayes - F1 Macro score: {:.2f}".format(best_nb_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/bow-balance-naive-bayes.csv', index=False)

bow balanced SGD

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample
from sklearn.linear_model import SGDClassifier

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load dataset
import pandas as pd
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    return " ".join(filtered_text)

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()


majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

# Upsample minority class
minority_upsampled = resample(minority_class,replace=True,n_samples=len(majority_class),random_state=35)


text_df = pd.concat([majority_class, minority_upsampled])

# Vectorize
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
sgd_model = SGDClassifier(loss='log_loss', random_state=35)
sgd_model.fit(x_train, y_train)
sgd_pred = sgd_model.predict(x_test)
sgd_acc = accuracy_score(y_test, sgd_pred)
f1_macro = f1_score(y_test, sgd_pred, average='macro')

print("Test accuracy: {:.2f}%".format(sgd_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, sgd_pred))
print("\n")
print(classification_report(y_test, sgd_pred))

# Hyperparameter tuning
param_grid = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4]
}

grid = GridSearchCV(SGDClassifier(loss='log_loss'), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)
print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

sgd_acc_best = accuracy_score(y_test, y_pred)
print("Best SGD - Test accuracy: {:.2f}%".format(sgd_acc_best * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_sgd_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best SGD - F1 Macro score: {:.2f}".format(best_sgd_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/bow-balance-sgd.csv', index=False)

bow balanced svm

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample

In [ ]:

# Mount Google Drive
drive.mount('/content/drive/')

# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [normalize_word(w) for w in text_tokens if w not in stop_words]
    return " ".join(filtered_text)

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)

def stemming(data):
    return " ".join([stemmer.stem(word) for word in data.split()])

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

# Upsample minority class
minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class),random_state=35)
text_df = pd.concat([majority_class, minority_upsampled])

# Vectorize
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
SVCmodel = LinearSVC(class_weight='balanced')
SVCmodel.fit(x_train, y_train)
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(y_test, svc_pred)
f1_macro = f1_score(y_test, svc_pred, average='macro')

print("Test accuracy: {:.2f}%".format(svc_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],
}

grid = GridSearchCV(LinearSVC(class_weight='balanced'), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)
print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Voting Classifier - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(lambda x: stemming(x))
evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/bow-balance-svm.csv', index=False)

tfidf balanced LR

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load dataset
import pandas as pd
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [normalize_word(w) for w in text_tokens if w not in stop_words]
    return " ".join(filtered_text)

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)

def stemming(data):
    return " ".join([stemmer.stem(word) for word in data.split()])

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

# Upsample minority class
minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class),random_state=35)
text_df = pd.concat([majority_class, minority_upsampled])

# TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
logreg_model = LogisticRegression(class_weight='balanced', solver='liblinear')  # Set class_weight to balanced
logreg_model.fit(x_train, y_train)
logreg_pred = logreg_model.predict(x_test)
logreg_acc = accuracy_score(y_test, logreg_pred)
f1_macro = f1_score(y_test, logreg_pred, average='macro')

print("Test accuracy: {:.2f}%".format(logreg_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],
}

grid = GridSearchCV(LogisticRegression(class_weight='balanced', solver='liblinear'), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)
print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Logistic Regression - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))
best_logreg_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Logistic Regression - F1 Macro score: {:.2f}".format(best_logreg_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(lambda x: stemming(x))

evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/tfidf-balance-logistic-regression.csv', index=False)

tfidf balanced naive bayse

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load dataset
import pandas as pd
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    return " ".join(filtered_text)

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

# upsampling
majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

minority_upsampled = resample(minority_class,replace=True, n_samples=len(majority_class), random_state=35)

text_df = pd.concat([majority_class, minority_upsampled])

# TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)

nb_pred = nb_model.predict(x_test)
nb_acc = accuracy_score(y_test, nb_pred)
f1_macro = f1_score(y_test, nb_pred, average='macro')

print("Test accuracy: {:.2f}%".format(nb_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, nb_pred))
print("\n")
print(classification_report(y_test, nb_pred))

# Hyperparameter tuning
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5]
}

grid = GridSearchCV(MultinomialNB(), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)
print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

nb_acc = accuracy_score(y_test, y_pred)
print("Best Naive Bayes - Test accuracy: {:.2f}%".format(nb_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_nb_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Naive Bayes - F1 Macro score: {:.2f}".format(best_nb_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
evaluation_test = vectorizer.transform(eval_df['text'])

predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/tfidf-balance-naive-bayes.csv', index=False)

tfidf balanced SGD

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample
from sklearn.linear_model import SGDClassifier


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load dataset
import pandas as pd
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    return " ".join(filtered_text)

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

# upsampling
majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]
minority_upsampled = resample(minority_class,replace=True,n_samples=len(majority_class),random_state=35)
text_df = pd.concat([majority_class, minority_upsampled])

# tfidf
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)


sgd_model = SGDClassifier(loss='log_loss', random_state=35)
sgd_model.fit(x_train, y_train)

# Predictions and evaluation
sgd_pred = sgd_model.predict(x_test)
sgd_acc = accuracy_score(y_test, sgd_pred)
f1_macro = f1_score(y_test, sgd_pred, average='macro')

print("Test accuracy: {:.2f}%".format(sgd_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, sgd_pred))
print("\n")
print(classification_report(y_test, sgd_pred))

# Hyperparameter tuning
param_grid = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4]
}

grid = GridSearchCV(SGDClassifier(loss='log_loss'), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

sgd_acc_best = accuracy_score(y_test, y_pred)
print("Best SGD - Test accuracy: {:.2f}%".format(sgd_acc_best * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_sgd_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best SGD - F1 Macro score: {:.2f}".format(best_sgd_f1_macro))

#data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/tfidf-balance-sgd.csv', index=False)

tfidf balanced svm

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.utils import resample

In [ ]:

# Mount Google Drive
drive.mount('/content/drive/')

# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)

# Pre-processing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [normalize_word(w) for w in text_tokens if w not in stop_words]
    return " ".join(filtered_text)

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)

def stemming(data):
    return " ".join([stemmer.stem(word) for word in data.split()])

# Train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

# Visualizations
plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

# upsampling for handle class imbalance
majority_class = text_df[text_df['sentiment'] == 1]
minority_class = text_df[text_df['sentiment'] == 0]

# Upsample minority class
minority_upsampled = resample(minority_class,
                               replace=True,
                               n_samples=len(majority_class),
                               random_state=35)

text_df = pd.concat([majority_class, minority_upsampled])

# Vectorize
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

# Train LinearSVC
SVCmodel = LinearSVC(class_weight='balanced')  # Set class_weight to balanced
SVCmodel.fit(x_train, y_train)

svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(y_test, svc_pred)
f1_macro = f1_score(y_test, svc_pred, average='macro')

print("Test accuracy: {:.2f}%".format(svc_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))
print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],
}

grid = GridSearchCV(LinearSVC(class_weight='balanced'), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Voting Classifier - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))

# data processing on evaluation set
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(lambda x: stemming(x))

# Vectorize evaluation  data
evaluation_test = vectorizer.transform(eval_df['text'])


predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/tfidf-balance-svm.csv', index=False)

bow+tfidf svm

In [ ]:
import pandas as pd
import numpy as np
import re
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE



In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# pre-processing Step
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)



# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))


fig = plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)

fig = plt.figure(figsize=(7, 7))
colors = ("yellowgreen", "red")
wp = {'linewidth': 2, 'edgecolor': "black"}
tags = text_df['sentiment'].value_counts()
explode = (0.1, 0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors=colors,
          startangle=90, wedgeprops=wp, explode=explode, label='')
plt.title('Distribution of sentiments')

#  WordCloud for positive tweets
pos_tweets = text_df[text_df.sentiment == 1]
text = ' '.join([word for word in pos_tweets['text']])
plt.figure(figsize=(20, 15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in positive tweets', fontsize=19)
plt.show()

#  WordCloud for negative tweets
neg_tweets = text_df[text_df.sentiment == 0]
text = ' '.join([word for word in neg_tweets['text']])
plt.figure(figsize=(20, 15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in negative tweets', fontsize=19)
plt.show()


count_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=15000)
X_tf = count_vectorizer.fit_transform(text_df['text'])

# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=15000)
X_tfidf = tfidf_vectorizer.fit_transform(text_df['text'])

X = hstack([X_tf, X_tfidf])
Y = text_df['sentiment']
#train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(y_test, svc_pred)
print("Test accuracy: {:.2f}%".format(svc_acc * 100))
f1_macro = f1_score(y_test, svc_pred, average='macro')
print("F1 Macro score: {:.2f}".format(f1_macro))

print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'class_weight': [None, 'balanced'],
}

# Perform grid search
grid = GridSearchCV(LinearSVC(), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Voting Classifier - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))


#  data processing on evaluation set
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(lambda x: stemming(x))


best_model.fit(X, Y)


eval_tf = count_vectorizer.transform(eval_df['text'])
eval_tfidf = tfidf_vectorizer.transform(eval_df['text'])

evaluation_test = hstack([eval_tf, eval_tfidf])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/combined-svm.csv', index=False)

bow+tfidf naive bayse

In [ ]:
import pandas as pd
import numpy as np
import re
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.naive_bayes import MultinomialNB
from google.colab import drive




In [ ]:
drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)


development_df.head()

development_df.info()

development_df.isnull().sum()

development_df.columns

text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df.head()

print(text_df['text'].iloc[0], "\n")
print(text_df['text'].iloc[1], "\n")
print(text_df['text'].iloc[2], "\n")
print(text_df['text'].iloc[3], "\n")
print(text_df['text'].iloc[4], "\n")


# Data processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming function
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))





text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))
text_df.head()


fig = plt.figure(figsize=(5,5))
sns.countplot(x='sentiment', data=text_df)


fig = plt.figure(figsize=(7,7))
colors = ("yellowgreen",  "red")
wp = {'linewidth':2, 'edgecolor':"black"}
tags = text_df['sentiment'].value_counts()
explode = (0.1, 0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors=colors, startangle=90, wedgeprops=wp, explode=explode, label='')
plt.title('Distribution of sentiments')

# Word cloud for positive tweets
pos_tweets = text_df[text_df.sentiment == 1]
text = ' '.join([word for word in pos_tweets['text']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in positive tweets', fontsize=19)
plt.show()

# Word cloud for negative tweets
neg_tweets = text_df[text_df.sentiment == 0]
text = ' '.join([word for word in neg_tweets['text']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in negative tweets', fontsize=19)
plt.show()


count_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=15000)
X_tf = count_vectorizer.fit_transform(text_df['text'])

# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=15000)
X_tfidf = tfidf_vectorizer.fit_transform(text_df['text'])

X = hstack([X_tf, X_tfidf])

Y = text_df['sentiment']


# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)
nb_pred = nb_model.predict(x_test)
nb_acc = accuracy_score(nb_pred, y_test)
nb_f1_macro = f1_score(y_test, nb_pred, average='macro')
print("Test accuracy: {:.2f}%".format(nb_acc * 100))
print("F1 Macro score: {:.2f}".format(nb_f1_macro))

print(confusion_matrix(y_test, nb_pred))
print("\n")
print(classification_report(y_test, nb_pred))

style.use('classic')
cm = confusion_matrix(y_test, nb_pred, labels=nb_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nb_model.classes_)
disp.plot()

# Grid Search for hyperparameter tuning
param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}
grid = GridSearchCV(MultinomialNB(), param_grid,n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)
nb_acc = accuracy_score(y_pred, y_test)
nb_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy after Grid Search: {:.2f}%".format(nb_acc * 100))
print("F1 Macro score after Grid Search: {:.2f}".format(nb_f1_macro))

print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))



#  data processing
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)


best_model.fit(X, Y)

# Vectorize evaluation text data
eval_tf = count_vectorizer.transform(eval_df['text'])
eval_tfidf = tfidf_vectorizer.transform(eval_df['text'])

evaluation_test = hstack([eval_tf, eval_tfidf])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/combined-naivebayse.csv', index=False)

bow+tfidf logistic-regression

In [ ]:
import pandas as pd
import numpy as np
import re
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from google.colab import drive
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay


In [ ]:
# Mount Google Drive

drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming function
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


print(development_df.head())
print(development_df.info())
print(development_df.isnull().sum())


text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(stemming)

fig = plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

# Create word clouds
def create_wordcloud(text, title):
    plt.figure(figsize=(10, 8))
    wordcloud = WordCloud(max_words=100, background_color='white').generate(text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()

# Positive tweets
pos_tweets = text_df[text_df.sentiment == 1]
create_wordcloud(' '.join(pos_tweets['text']), 'Most Frequent Words in Positive Tweets')

# Negative tweets
neg_tweets = text_df[text_df.sentiment == 0]
create_wordcloud(' '.join(neg_tweets['text']), 'Most Frequent Words in Negative Tweets')

# Vectorization
count_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=15000)
X_tf = count_vectorizer.fit_transform(text_df['text'])

# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=15000)
X_tfidf = tfidf_vectorizer.fit_transform(text_df['text'])

X = hstack([X_tf, X_tfidf])
Y = text_df['sentiment']

# train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

logreg = LogisticRegression(max_iter=1000, class_weight='balanced')
logreg.fit(x_train, y_train)

logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(y_test, logreg_pred)
f1_macro = f1_score(y_test, logreg_pred, average='macro')
print("Test accuracy: {:.2f}%".format(logreg_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))

print(confusion_matrix(y_test, logreg_pred))
print(classification_report(y_test, logreg_pred))


cm = confusion_matrix(y_test, logreg_pred, labels=logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logreg.classes_)
disp.plot()

# Hyperparameter tuning with Grid Search
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)

best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Voting Classifier - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
best_voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))

#  data processing on the evaluation set
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)

best_model.fit(X, Y)

# Vectorize evaluation  data
eval_tf = count_vectorizer.transform(eval_df['text'])
eval_tfidf = tfidf_vectorizer.transform(eval_df['text'])


evaluation_test = hstack([eval_tf, eval_tfidf])

predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/combined-logisticRegression.csv', index=False)

bow+tfidf  sgd

In [ ]:
import pandas as pd
import numpy as np
import re
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
import warnings
from sklearn.linear_model import SGDClassifier


In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)


# pre-processing Step
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)



# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


# data processing
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

# Vectorize
count_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=15000)
X_tf = count_vectorizer.fit_transform(text_df['text'])

# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=15000)
X_tfidf = tfidf_vectorizer.fit_transform(text_df['text'])

X = hstack([X_tf, X_tfidf])
Y = text_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

sgd_clf = SGDClassifier(loss='hinge', random_state=35)
sgd_clf.fit(x_train, y_train)
y_pred = sgd_clf.predict(x_test)
sgd_acc = accuracy_score(y_test, y_pred)
sgd_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy: {:.2f}%".format(sgd_acc * 100))
print("F1 Macro score: {:.2f}".format(sgd_f1_macro))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))
style.use('classic')
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

param_grid = {
    'loss': ['hinge', 'log_loss', 'squared_hinge', 'modified_huber'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000]
}

grid_search = GridSearchCV(estimator=sgd_clf, param_grid=param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid_search.fit(x_train, y_train)

print("Best parameters from Grid Search:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(x_test)
best_sgd_acc = accuracy_score(y_test, y_pred_best)
best_sgd_f1_macro = f1_score(y_test, y_pred_best, average='macro')

print("Best SGD Classifier - Test accuracy: {:.2f}%".format(best_sgd_acc * 100))
print("Best SGD Classifier - F1 Macro score: {:.2f}".format(best_sgd_f1_macro))
cm_best = confusion_matrix(y_test, y_pred_best)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best)
disp_best.plot()

# data processing to evaluation dataset
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)

best_model.fit(X, Y)


eval_tf = count_vectorizer.transform(eval_df['text'])
eval_tfidf = tfidf_vectorizer.transform(eval_df['text'])


evaluation_test = hstack([eval_tf, eval_tfidf])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/combined-sgd.csv', index=False)

tfidf xgboost

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from google.colab import drive
import joblib
from joblib import Parallel, delayed
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score

In [ ]:

drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in text_df['text'])
text_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in text_df['text'] )

# vectorizer = CountVectorizer(max_features=2048)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=2048)  # Reduced max_features
X = vectorizer.fit_transform(development_df['text'])
Y = development_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

xgb_model = xgb.XGBClassifier(max_depth=6, random_state=42, n_estimators=1000, use_label_encoder=False, eval_metric='mlogloss', nthread=3)
xgb_model.fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)

xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_f1_macro = f1_score(y_test, xgb_pred, average='macro')
print("Test accuracy: {:.2f}%".format(xgb_acc * 100))
print("F1 Macro score: {:.2f}".format(xgb_f1_macro))
print(confusion_matrix(y_test, xgb_pred))
print("\nClassification Report:\n", classification_report(y_test, xgb_pred))

cm = confusion_matrix(y_test, xgb_pred, labels=xgb_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb_model.classes_)
disp.plot()

# Random Search for hyperparameter tuning
param_dist = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [5, 6, 7, 8],
}
random_search = RandomizedSearchCV(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss', nthread=3),
                                   param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=35)
random_search.fit(x_train, y_train)

print("Best parameters:", random_search.best_params_)


best_model = random_search.best_estimator_
y_pred = best_model.predict(x_test)

xgb_acc = accuracy_score(y_test, y_pred)
xgb_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy after Random Search: {:.2f}%".format(xgb_acc * 100))
print("F1 Macro score after Random Search: {:.2f}".format(xgb_f1_macro))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Data processing on evaluation data
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in eval_df['text'])
eval_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in eval_df['text'] )

evaluation_test = vectorizer.transform(evaluation_df['text'])
best_model.fit(X, Y)


predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/fulldata-tfidf-xgboost.csv', index=False)

tfidf Voting Classifier(Logistic Regression+ Stochastic Gradient Descent)

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.naive_bayes import MultinomialNB
from google.colab import drive



In [ ]:
drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))




#  data processing
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))


# vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorizer=TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
logreg = LogisticRegression(random_state=42,max_iter=10000)
sgd = SGDClassifier(random_state=42,loss='log_loss',max_iter=10000)
voting_clf = VotingClassifier(estimators=[('logreg', logreg), ('sgd', sgd)], voting='soft')
voting_clf.fit(x_train, y_train)
y_pred = voting_clf.predict(x_test)

voting_acc = accuracy_score(y_test, y_pred)
voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy: {:.2f}%".format(voting_acc * 100))
print("F1 Macro score: {:.2f}".format(voting_f1_macro))


print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))


style.use('classic')
cm = confusion_matrix(y_test, y_pred, labels=voting_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=voting_clf.classes_)
disp.plot()



# parameter grid for Grid Search
param_grid = {
    'logreg__C': [0.001, 0.01, 0.1, 1, 10],
    'sgd__alpha': [0.0001, 0.001, 0.01, 0.1]
}

grid_search = GridSearchCV(estimator=voting_clf, param_grid=param_grid, scoring='f1_macro', cv=3)
grid_search.fit(x_train, y_train)

print("Best parameters from Grid Search:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(x_test)
best_voting_acc = accuracy_score(y_test, y_pred_best)
best_voting_f1_macro = f1_score(y_test, y_pred_best, average='macro')

print("Best Voting Classifier - Test accuracy: {:.2f}%".format(best_voting_acc * 100))
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))

cm_best = confusion_matrix(y_test, y_pred_best)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best, display_labels=best_model.classes_)
disp_best.plot()


# data processing
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)

best_model.fit(X, Y)

eveluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(eveluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('fulldata-tfidf-LrSgd.csv', index=False)

tfidf svm

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE

In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# pre-processing Step
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)



# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


# train data
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))


fig = plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)

fig = plt.figure(figsize=(7, 7))
colors = ("yellowgreen", "red")
wp = {'linewidth': 2, 'edgecolor': "black"}
tags = text_df['sentiment'].value_counts()
explode = (0.1, 0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors=colors,
          startangle=90, wedgeprops=wp, explode=explode, label='')
plt.title('Distribution of sentiments')

#  WordCloud for positive tweets
pos_tweets = text_df[text_df.sentiment == 1]
text = ' '.join([word for word in pos_tweets['text']])
plt.figure(figsize=(20, 15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in positive tweets', fontsize=19)
plt.show()

#  WordCloud for negative tweets
neg_tweets = text_df[text_df.sentiment == 0]
text = ' '.join([word for word in neg_tweets['text']])
plt.figure(figsize=(20, 15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in negative tweets', fontsize=19)
plt.show()

# Vectorize
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=15000)
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']
#train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(y_test, svc_pred)
print("Test accuracy: {:.2f}%".format(svc_acc * 100))
f1_macro = f1_score(y_test, svc_pred, average='macro')
print("F1 Macro score: {:.2f}".format(f1_macro))

print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'class_weight': [None, 'balanced'],
}

# Perform grid search
grid = GridSearchCV(LinearSVC(), param_grid, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameter:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Voting Classifier - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

best_voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))


#  data processing on evaluation set
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(lambda x: stemming(x))


best_model.fit(X, Y)

evaluation_test = vectorizer.transform(eval_df['text'])

predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('fulldata_tfidf-linearSVM.csv', index=False)

tfidf sgd

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
import warnings
from sklearn.linear_model import SGDClassifier

In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)


# pre-processing Step
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)



# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


# data processing
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

#  TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

sgd_clf = SGDClassifier(loss='hinge', random_state=35)
sgd_clf.fit(x_train, y_train)
y_pred = sgd_clf.predict(x_test)


sgd_acc = accuracy_score(y_test, y_pred)
sgd_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy: {:.2f}%".format(sgd_acc * 100))
print("F1 Macro score: {:.2f}".format(sgd_f1_macro))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))
style.use('classic')
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

param_grid = {
    'loss': ['hinge', 'log_loss', 'squared_hinge', 'modified_huber'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000]
}

grid_search = GridSearchCV(estimator=sgd_clf, param_grid=param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid_search.fit(x_train, y_train)

print("Best parameters from Grid Search:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(x_test)
best_sgd_acc = accuracy_score(y_test, y_pred_best)
best_sgd_f1_macro = f1_score(y_test, y_pred_best, average='macro')

print("Best SGD Classifier - Test accuracy: {:.2f}%".format(best_sgd_acc * 100))
print("Best SGD Classifier - F1 Macro score: {:.2f}".format(best_sgd_f1_macro))

cm_best = confusion_matrix(y_test, y_pred_best)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best)
disp_best.plot()

eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)

best_model.fit(X, Y)

evaluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/full-tfidf-sgd1.csv', index=False)

tfidf random forest

In [ ]:
import pandas as pd
import numpy as np
import re
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from google.colab import drive
import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score

In [ ]:

drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming function
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]  t
    return " ".join(text)
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in text_df['text'])
text_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in text_df['text'] )

# vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=4096)
# vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=4096)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(x_train, y_train)
rf_pred = rf_model.predict(x_test)
print(f"Test accuracy: {accuracy_score(y_test, rf_pred) * 100:.2f}%")
print(f"F1 Macro score: {f1_score(y_test, rf_pred, average='macro'):.2f}")
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

# Grid Search for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)

best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

print(f"Test accuracy after Grid Search: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(f"F1 Macro score after Grid Search: {f1_score(y_test, y_pred, average='macro'):.2f}")

# Data processing on evaluation data
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in eval_df['text'])
eval_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in eval_df['text'] )

evaluation_test = vectorizer.transform(eval_df['text'])
best_model.fit(X, Y)
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/fulldata-tfidf-randomforest4096.csv', index=False)


tfidf naive bayse

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.naive_bayes import MultinomialNB
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)


development_df.head()

development_df.info()

development_df.isnull().sum()

development_df.columns

text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df.head()

print(text_df['text'].iloc[0], "\n")
print(text_df['text'].iloc[1], "\n")
print(text_df['text'].iloc[2], "\n")
print(text_df['text'].iloc[3], "\n")
print(text_df['text'].iloc[4], "\n")


# Data processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))




text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))
text_df.head()

fig = plt.figure(figsize=(5,5))
sns.countplot(x='sentiment', data=text_df)

fig = plt.figure(figsize=(7,7))
colors = ("yellowgreen",  "red")
wp = {'linewidth':2, 'edgecolor':"black"}
tags = text_df['sentiment'].value_counts()
explode = (0.1, 0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors=colors, startangle=90, wedgeprops=wp, explode=explode, label='')
plt.title('Distribution of sentiments')

# Word cloud for positive tweets
pos_tweets = text_df[text_df.sentiment == 1]
text = ' '.join([word for word in pos_tweets['text']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in positive tweets', fontsize=19)
plt.show()

# Word cloud for negative tweets
neg_tweets = text_df[text_df.sentiment == 0]
text = ' '.join([word for word in neg_tweets['text']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in negative tweets', fontsize=19)
plt.show()

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vect = vectorizer.fit(text_df['text'])

feature_names = vectorizer.get_feature_names_out()
print("Number of features: {}\n".format(len(feature_names)))
print("First 20 features:\n {}".format(feature_names[:20]))

X = text_df['text']
Y = text_df['sentiment']
X = vect.transform(X)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)
nb_pred = nb_model.predict(x_test)
nb_acc = accuracy_score(nb_pred, y_test)
nb_f1_macro = f1_score(y_test, nb_pred, average='macro')
print("Test accuracy: {:.2f}%".format(nb_acc * 100))
print("F1 Macro score: {:.2f}".format(nb_f1_macro))
print(confusion_matrix(y_test, nb_pred))
print("\n")
print(classification_report(y_test, nb_pred))
style.use('classic')
cm = confusion_matrix(y_test, nb_pred, labels=nb_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nb_model.classes_)
disp.plot()

# Grid Search for hyperparameter tuning
param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}
grid = GridSearchCV(MultinomialNB(), param_grid,n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

nb_acc = accuracy_score(y_pred, y_test)
nb_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy after Grid Search: {:.2f}%".format(nb_acc * 100))
print("F1 Macro score after Grid Search: {:.2f}".format(nb_f1_macro))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))



#  data processing
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)
best_model.fit(X, Y)
eveluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(eveluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('full-tfidf-naivebayse.csv', index=False)

tfidf logistic-regression

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from google.colab import drive
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay


In [ ]:
# Mount Google Drive

drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

def normalize_word(word):
     return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

print(development_df.head())
print(development_df.info())
print(development_df.isnull().sum())
text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(stemming)


fig = plt.figure(figsize=(5, 5))
sns.countplot(x='sentiment', data=text_df)
plt.title('Sentiment Distribution')
plt.show()

#  word clouds
def create_wordcloud(text, title):
    plt.figure(figsize=(10, 8))
    wordcloud = WordCloud(max_words=100, background_color='white').generate(text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()

# Positive tweets
pos_tweets = text_df[text_df.sentiment == 1]
create_wordcloud(' '.join(pos_tweets['text']), 'Most Frequent Words in Positive Tweets')

# Negative tweets
neg_tweets = text_df[text_df.sentiment == 0]
create_wordcloud(' '.join(neg_tweets['text']), 'Most Frequent Words in Negative Tweets')

# Vectorization using TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = tfidf_vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

logreg = LogisticRegression(max_iter=1000, class_weight='balanced')
logreg.fit(x_train, y_train)

logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(y_test, logreg_pred)
f1_macro = f1_score(y_test, logreg_pred, average='macro')
print("Test accuracy: {:.2f}%".format(logreg_acc * 100))
print("F1 Macro score: {:.2f}".format(f1_macro))

print(confusion_matrix(y_test, logreg_pred))
print(classification_report(y_test, logreg_pred))

cm = confusion_matrix(y_test, logreg_pred, labels=logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logreg.classes_)
disp.plot()

# Hyperparameter tuning with Grid Search
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, n_jobs=-1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

logreg_acc = accuracy_score(y_test, y_pred)
print("Best Voting Classifier - Test accuracy: {:.2f}%".format(logreg_acc * 100))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
best_voting_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Best Voting Classifier - F1 Macro score: {:.2f}".format(best_voting_f1_macro))

#  data processing on the evaluation set
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)
best_model.fit(X, Y)
evaluation_test = tfidf_vectorizer.transform(eval_df['text'])
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/tfidf-logisticRegression_full_data_predictions.csv', index=False)


tfidf Gradient Boosting Machine

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from google.colab import drive
import joblib
from joblib import Parallel, delayed
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score

In [ ]:

drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in text_df['text'])
text_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in text_df['text'] )


# vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=4096)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=4096)
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

gb_clf = GradientBoostingClassifier(random_state=35)
gb_clf.fit(x_train, y_train)
y_pred = gb_clf.predict(x_test)

gb_acc = accuracy_score(y_test, y_pred)
gb_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy: {:.2f}%".format(gb_acc * 100))
print("F1 Macro score: {:.2f}".format(gb_f1_macro))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))
style.use('classic')
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Grid Search for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'max_depth': [3, 5, 7, 9]
}

grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, scoring='f1_macro', cv=3,n_jobs=-1)
grid_search.fit(x_train, y_train)

print("Best parameters from Grid Search:", grid_search.best_params_)


best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(x_test)

best_gb_acc = accuracy_score(y_test, y_pred_best)
best_gb_f1_macro = f1_score(y_test, y_pred_best, average='macro')

print("Best Gradient Boosting Classifier - Test accuracy: {:.2f}%".format(best_gb_acc * 100))
print("Best Gradient Boosting Classifier - F1 Macro score: {:.2f}".format(best_gb_f1_macro))

cm_best = confusion_matrix(y_test, y_pred_best)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best)
disp_best.plot()

# Data processing on evaluation data
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in eval_df['text'])
eval_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in eval_df['text'] )
eveluation_test = vectorizer.transform(eval_df['text'])
best_model.fit(X, Y)
predictions = best_model.predict(eveluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/fulldata-tfidf-gbm.csv', index=False)

tfidf extra tree

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.naive_bayes import MultinomialNB
from google.colab import drive
from joblib import Parallel, delayed

In [ ]:
drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')


def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in text_df['text'])
text_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in text_df['text'] )

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=2048)
X = vectorizer.fit_transform(text_df['text'])
Y = text_df['sentiment']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)
extra_tree_clf = ExtraTreesClassifier(random_state=35, n_jobs=-1)

# Hyperparameter tuning with RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rand_search = RandomizedSearchCV(estimator=extra_tree_clf, param_distributions=param_distributions,
                                 scoring='f1_macro', n_iter=10, cv=3, n_jobs=-1, random_state=35)
rand_search.fit(x_train, y_train)

print("Best parameters from Randomized Search:", rand_search.best_params_)
best_model = rand_search.best_estimator_
y_pred_best = best_model.predict(x_test)
best_extra_tree_acc = accuracy_score(y_test, y_pred_best)
best_extra_tree_f1_macro = f1_score(y_test, y_pred_best, average='macro')

print("Best Extra Trees Classifier - Test accuracy: {:.2f}%".format(best_extra_tree_acc * 100))
print("Best Extra Trees Classifier - F1 Macro score: {:.2f}".format(best_extra_tree_f1_macro))

print(confusion_matrix(y_test, y_pred_best))
print("\n")
print(classification_report(y_test, y_pred_best))

cm_best = confusion_matrix(y_test, y_pred_best)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best)
disp_best.plot()
plt.title("Best Model Confusion Matrix")
plt.show()

# Data processing on evaluation data
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = Parallel(n_jobs=-1)(delayed(data_processing)(text) for text in eval_df['text'])
eval_df['text'] = Parallel(n_jobs=-1)( delayed(stemming)(tokens) for tokens in eval_df['text'] )

evaluation_test = vectorizer.transform(eval_df['text'])

best_model.fit(X, Y)
predictions = best_model.predict(evaluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('/content/drive/MyDrive/fulldata-tfidf-etc.csv', index=False)

tfidf decision tree

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.naive_bayes import MultinomialNB
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')
# Load dataset
development_path = "/content/drive/MyDrive/dsl-fall/development.csv"
development_df = pd.read_csv(development_path)
evaluation_path = "/content/drive/MyDrive/dsl-fall/evaluation.csv"
evaluation_df = pd.read_csv(evaluation_path)

# Data processing function
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    text_tokens = [normalize_word(w) for w in text_tokens]
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)


# # Function to clean and preprocess the text
# def data_processing(text):
#     # Remove HTML entities
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text()

#     # Remove mentions
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)

#     # Remove URLs
#     text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

#     # Remove punctuation and numbers and handle unnecessary whitespaces
#     text = re.sub(r'[^a-zA-Z\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Tokenize the text
#     words = text.split()
#     # Remove stop words and meaningless words
#     meaningless_words = {'&lt', '&quot', '&amp', '&gt'}  # Add more if necessary
#     processed_words = []
#     for word in words:
#         normalized_word = normalize_word(word.lower())
#         if normalized_word not in stop_words and normalized_word not in meaningless_words:
#             # Apply stemming and lemmatization
#             stemmed_word = stemmer.stem(normalized_word)
#             lemmatized_word = lemmatizer.lemmatize(stemmed_word)
#             processed_words.append(lemmatized_word)

#     return " ".join(processed_words)



# Stemming
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data.split()]
    return " ".join(text)

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


text_df = development_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
text_df['text'] = text_df['text'].apply(data_processing)
text_df['text'] = text_df['text'].apply(lambda x: stemming(x))

# Vectorize text data
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
# CountVectorizer(ngram_range=(1,2),max_features=4096)
vect = vectorizer.fit(text_df['text'])


X = text_df['text']
Y = text_df['sentiment']
X = vect.transform(X)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

dt_model = DecisionTreeClassifier(random_state=35)
dt_model.fit(x_train, y_train)
dt_pred = dt_model.predict(x_test)
dt_acc = accuracy_score(dt_pred, y_test)
dt_f1_macro = f1_score(y_test, dt_pred, average='macro')
print("Test accuracy: {:.2f}%".format(dt_acc * 100))
print("F1 Macro score: {:.2f}".format(dt_f1_macro))
print(confusion_matrix(y_test, dt_pred))
print("\n")
print(classification_report(y_test, dt_pred))
style.use('classic')
cm = confusion_matrix(y_test, dt_pred, labels=dt_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dt_model.classes_)
disp.plot()

# Grid Search for hyperparameter tuning
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)

dt_acc = accuracy_score(y_pred, y_test)
dt_f1_macro = f1_score(y_test, y_pred, average='macro')
print("Test accuracy after Grid Search: {:.2f}%".format(dt_acc * 100))
print("F1 Macro score after Grid Search: {:.2f}".format(dt_f1_macro))
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))
#  data processing
eval_df = evaluation_df.drop(['ids', 'date', 'flag', 'user'], axis=1)
eval_df['text'] = eval_df['text'].apply(data_processing)
eval_df['text'] = eval_df['text'].apply(stemming)

best_model.fit(X, Y)

eveluation_test = vectorizer.transform(eval_df['text'])
predictions = best_model.predict(eveluation_test)

# Save predictions to CSV
submission = pd.DataFrame({'Id': evaluation_df['ids'], 'Predicted': predictions})
submission.to_csv('bow-DT.csv', index=False)